In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from model import T5ModelForPreTraining

/usr/local/lib/python3.9/dist-packages/horovod/common/util.py:258: UserWarning: Framework pytorch installed with version 1.10.2+cu111 but found version 1.10.1+cu111.
             This can result in unexpected behavior including runtime errors.
             Reinstall Horovod using `pip install --no-cache-dir` to build with the new version.
  warnings.warn(get_version_mismatch_message(name, version, installed_version))
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zer

In [2]:
config = PeftConfig.from_pretrained("augustinLib/KET5-Prefix-Tuning-Lbox")
model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR/ke-t5-large")
model = PeftModel.from_pretrained(model, "augustinLib/KET5-Prefix-Tuning-Lbox")


tokenizer = AutoTokenizer.from_pretrained("KETI-AIR/ke-t5-large")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [3]:
model.print_trainable_parameters()

trainable params: 12,582,912 || all params: 795,601,920 || trainable%: 1.5815587775353785


In [4]:
a = tokenizer.get_vocab()
a["."]

3

In [5]:
model.to("cuda:0")
inputs = tokenizer(
    "피고인은 2021. 5. 23. 22:50경 서울 영등포구 B에 있는 ‘C매장’ 앞에서, 술 취한 남성 2명이 싸우고 있다는 112신고를 받고 출동한 서울영등포경찰서 D지구대 소속 경장 E(남, 33세)이 일행과 몸싸움을 하는 피고인을 제지한 후 귀가를 요구하자, “경찰이 책임져야지”라고 소리를 지르며 위 경찰관의 오른쪽 가슴 부위를 주먹으로 1회 때려 폭행하였다. 이로써 피고인은 경찰공무원의 112신고 사건처리에 관한 정당한 직무집행을 방해하였다.",
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding="max_length",
)

In [6]:
inputs["input_ids"]

tensor([[55912,  1352, 20213,  1170,    11, 11760, 26764, 24904, 18003, 18005,
            11, 30723,   739, 30723, 24733,  1104,   997, 22662,    11,  7478,
            45,   153,  8139,  5884,  3299,  1712,   304,     3,  6776,  2000,
             4, 13920,    19,  9392,     3,    57,     3,   573,     3,   495,
         33930,   508,   203, 50654,   384,     9,    53,    33,   497,    17,
            37,     4, 11538,    45,   189,    50,  9115,   156,  4801, 43948,
          2655,   580,    42,   270, 14207,  8029,    24,  1352, 17502,    77,
            58, 15088,  8323,   479,  1955,   794,     4,  1338,  9115,    15,
          8496,  1352, 17502,    44, 18982,  1096,    21,  4801,    51,  4061,
         30723,   479, 10019,     3,  5324, 13920,    19,  1352, 20213,  1170,
            11, 11760, 26764, 24904, 18003, 18005,     6, 30723,    11,  7478,
            45,  6389,     3,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [7]:
with torch.no_grad():
    inputs = {k: v.to("cuda:0") for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=32, num_beams=4, temperature=1.5, top_k=10, top_p=0.9, length_penalty=0.65)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))


RuntimeError: CUDA out of memory. Tried to allocate 384.00 MiB (GPU 0; 11.76 GiB total capacity; 3.30 GiB already allocated; 248.44 MiB free; 3.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF